In [2]:
# Step 1: get all data
from main import setup
dataframes, effect_data = setup("", True)

### Strength Metric Calculation

To evaluate the strength of an ability, we incorporate several factors: damage, effects, cooldown, and area of effect (AoE). Each factor contributes to a final score that reflects the overall potency of the ability.

#### Variables:

- **$ \text{AvgDmg} $**: Average damage dealt by the ability (computed using dice).
- **$ \text{EffScore} $**: The total effect score, considering both the effect’s strength and duration.
- **$ \text{CDPenalty} $**: A cooldown penalty based on the cooldown duration.
- **$ \text{AoEArea} $**: The area of effect based on the type and range of the ability.
- **$ r_i $**: The rating (or strength) of an individual effect $ i $ based on a scale from 1 to 5.
- **$ d_i $**: The duration of effect $ i $ in rounds (default of 1 if not specified).

#### Formulas:

1. **Effect Score**:

   $$
   \text{EffScore} = \sum_{i} r_i \times d_i
   $$

   where:
   - $ r_i $ is the strength rating of effect $ i $.
   - $ d_i $ is the duration of effect $ i $ in rounds.

<br>

2. **Area of Effect (AoE) Calculation**:

   The calculation varies based on the type of AoE and is based on tiles (5x5):
   - For "radius" attacks (circular area):
     $$
     \text{AoEArea} = 2 * range^2 + 2 * range 
     $$
   - For "cone" attacks:
     $$
     \text{AoEArea} = range ** 2
     $$
   - For "line" attacks (assuming a width of 5 feet):
     $$
     \text{AoEArea} = \text{range} \times 5
     $$
   - For "self" or "single" attacks:
     $$
     \text{AoEArea} = 1
     $$

<br>

3. **Final Strength Metric**:

   $$
   \text{Strength Metric} = \left(\text{AvgDmg} + \text{EffScore}\right) \times \sqrt{\text{AoEArea}}
   $$

In [3]:
import math
import pandas as pd
import numpy as np
import re

# Step 2: Ability Rating
action_df = dataframes["actions"]
# Step 2.1: Calculate dice damage (average, max, min, sd)
def parse_dice(dice_str):
    """Parses dice in format 'XdY+Z', 'XdY-Z', or 'XdY' and returns a tuple (X, Y, Z)."""
    pattern = r'(\d+)d(\d+)([+-]\d+)?'
    match_obj = re.fullmatch(pattern, dice_str.replace(' ', ''))
    if match_obj:
        X = int(match_obj.group(1))
        Y = int(match_obj.group(2))
        Z = int(match_obj.group(3)) if match_obj.group(3) else 0
        return X, Y, Z
    else:
        raise ValueError(f"Invalid dice string: {dice_str}")

def parse_dice_average(dice_str):
    """Calculates the average value of a dice roll in the form XdY+Z."""
    X, Y, Z = parse_dice(dice_str)
    return (X * (Y + 1) / 2) + Z

def combined_damage_statistics(damage_entries):
    total_min_damage = 0
    total_max_damage = 0
    total_avg_damage = 0
    total_variance = 0

    for damage in damage_entries:
        if isinstance(damage, list):
            dice_str = damage[0]
        else:
            dice_str = damage
        X, Y, Z = parse_dice(dice_str)

        min_damage = X * 1 + Z
        max_damage = X * Y + Z
        avg_damage = (X * (Y + 1) / 2) + Z
        variance = X * (((Y ** 2) - 1) / 12)

        total_min_damage += min_damage
        total_max_damage += max_damage
        total_avg_damage += avg_damage
        total_variance += variance

    combined_std_dev = np.sqrt(total_variance)
    return total_min_damage, total_max_damage, total_avg_damage, combined_std_dev

# Step 2.2: Calculate the area of effect based on AoE type and range 
def calculate_aoe_area(aoe_type, range_value):
    # Convert range from feet to tiles (1 tile = 5 x 5 feet)
    range_in_tiles = int(int(range_value) / 5) if range_value else 0
    match aoe_type:
        case "radius":
            area = 2 * (range_in_tiles ** 2) + 2 * range_in_tiles
        case "cone":
            area = range_in_tiles ** 2
        case "line":
            area = range_in_tiles
        case "self":
            area = 1
        case _:
            area = 1  # Assume single-target has minimal area impact
    return area

# Step 2.3: Calculate effect score based on ratings and parameter
def get_multiplier(parameter):
    if isinstance(parameter, (int, float)):
        print(f"parameter {parameter} is an integer")
        return parameter
    elif isinstance(parameter, str):
        duration = parameter.strip()
        match duration:
            case d if d.endswith('r'):
                return int(d.rstrip('r'))
            case d if d.startswith('lv'):
                return int(d.lstrip('lv'))
            case "until freed":
                return 1  # Special case
            case d if re.fullmatch(r'\d+d\d+([+-]\d+)?', d):
                return parse_dice_average(d)
            case d if d.endswith('ft'):
                return int(d.rstrip('ft'))
            case _:
                return 1  # Default multiplier if parsing fails
    else:
        return 1  # Default multiplier

def calculate_effect_value(effect_name, rating, parameter):
    """Calculates the effect value based on effect name, rating, and duration."""
    multiplier = get_multiplier(parameter)
    match effect_name:
        case "heal":
            effect_value = rating + multiplier
            print(f"Calculating 'heal' effect: rating {rating} + average heal {multiplier} = {effect_value}")
            return effect_value
        case "grappled":
            # TODO adjust the special case
            if parameter == "until freed":
                effect_value = rating * 2
                print(f"Calculating 'grappled' effect with 'until freed': rating {rating} * 2 = {effect_value}")
            else:
                effect_value = rating * multiplier
                print(f"Calculating 'grappled' effect: rating {rating} * duration multiplier {multiplier} = {effect_value}")
            return effect_value
        case _:
            # Default calculation
            # for effects with range 
            if parameter.endswith('ft'):
                effect_value = rating + multiplier/5 # distance converted in tiles
                print(f"Calculating default effect '{effect_name}': rating {rating} + distance {multiplier} = {effect_value}")
            # for the rest    
            else:    
                effect_value = rating * multiplier
                print(f"Calculating default effect '{effect_name}': rating {rating} * multiplier {multiplier} = {effect_value}")
            return effect_value

def calculate_effect_score(effects):
    effect_score = 0
    for effect in effects:
        effect_name = effect[0]
        duration = effect[1] if len(effect) > 1 else 1
        rating = effect_data.get(effect_name, 0)  # Default to 0 if the effect is not found
        effect_value = calculate_effect_value(effect_name, rating, duration)
        effect_score += effect_value
    return effect_score

# Step 2.4: Calculate a final metric based on all factors
def calculate_strength_metric(row):
    damages = row['damage']
    min_dmg, max_dmg, avg_dmg, std_dmg = combined_damage_statistics(damages)
    damage_score = avg_dmg
    effect_score = calculate_effect_score(row['effects'])
    aoe_area = calculate_aoe_area(row['aoe'], row['range'])
    strength_metric = round((damage_score + effect_score) * (aoe_area ** 0.5))
    return strength_metric, max_dmg, avg_dmg


output_series = action_df.apply(calculate_strength_metric, axis=1)
strength_metrics, max_damages, avg_damages = zip(*output_series)
action_df['strength_metric'] = strength_metrics
action_df['max_dmg'] = max_damages
action_df['avg_dmg'] = avg_damages

action_df


Calculating default effect 'exhausted': rating 5 * multiplier 4 = 20
Calculating default effect 'prone': rating 3 * multiplier 1 = 3
Calculating default effect 'blinded': rating 3 * multiplier 1 = 3
Calculating default effect 'paralysed': rating 5 * multiplier 2 = 10
Calculating default effect 'bleeding': rating 5 * multiplier 1 = 5
Calculating default effect 'bleeding': rating 5 * multiplier 2 = 10
Calculating default effect 'bleeding': rating 5 * multiplier 2 = 10
Calculating default effect 'petrified': rating 5 * multiplier 3 = 15
Calculating default effect 'difficult_terrain': rating 2 * multiplier 20 = 40
Calculating default effect 'resistance_(all)': rating 4 * multiplier 1 = 4
Calculating default effect 'bleeding': rating 5 * multiplier 1 = 5
Calculating default effect 'petrified': rating 5 * multiplier 3 = 15
Calculating default effect 'escape': rating 3 + distance 30 = 9.0
Calculating default effect 'knockback': rating 3 + distance 20 = 7.0
Calculating default effect 'knockbac

,name,legendary,cooldown,range,aoe,hitbonus,savereq,damage,effects,strength_metric,max_dmg,avg_dmg
0,Abyssaler Nebelhauch,False,"[3, r]",70,radius,0,"[con, 19]","[[12d6, acid]]","[[exhausted, lv4]]",1271,72,42.0
1,Beilhieb,False,[],5,single,5,[],"[[2d8, slashing]]",[],9,16,9.0
2,Berserkersturm,False,"[1, d]",40,line,9,[],"[[3d8+6, slashing]]","[[prone, 1r]]",64,30,19.5
3,Blendgranate,False,"[1, d]",30,radius,0,"[con, 16]",[],"[[blinded, 1r]]",27,0,0.0
4,Blitz aus der Tiefe,False,"[3, r]",100,single,0,"[dex, 18]","[[2d10, lightning]]","[[paralysed, 2r]]",21,20,11.0
5,Blutige Rache,False,[],5,single,10,[],"[[2d10+6, slashing], [2d8, necrotic]]","[[bleeding, lv1]]",31,42,26.0
6,Blutiger Schlag,False,[],5,single,10,[],"[[2d10+6, slashing], [2d10+6, slashing]]",[],34,52,34.0
7,Blutwoge,False,"[3, r]",15,cone,0,"[dex, 17]","[[5d8, necrotic], [3d6, slashing]]","[[bleeding, lv2]]",129,58,33.0
8,Bohrschnabel,False,[],5,single,5,[],"[[1d6+2, piercing]]",[],6,8,5.5
9,Brutale Hiebe,False,[],5,single,10,[],"[[2d12+6, slashing], [2d12+6, slashing]]","[[bleeding, lv2]]",48,60,38.0


### Effect Value Calculation

Calculate the **effect value** based on the **effect name**:

1. **For 'heal':**

   $$
   \text{effect\_value} = \text{rating}(\text{'heal'}) + \text{average dice roll}
   $$

2. **For 'grappled':**

   - **If duration is 'until freed':**

     $$
     \text{effect\_value} = \text{rating}(\text{'grappled'}) \times 2
     $$

   - **Else:**

     $$
     \text{effect\_value} = \text{rating}(\text{'grappled'}) \times \text{duration}
     $$

3. **For other effects:**

   - **If duration ends with 'ft':**

     $$
     \text{effect\_value} = \text{rating}(\text{effect\_name}) + \frac{\text{distance}}{5}
     $$

     (Adds distance in tiles to the rating)

   - **Else:**

     $$
     \text{effect\_value} = \text{rating}(\text{effect\_name}) \times \text{duration or level}
     $$

In [4]:
# Step 3: calculate strength of enemy without the actions
enemies_df = dataframes["enemies"]

def calculate_movement_metric(movement):
    movement_multipliers = {
        'laufend': 1.0,
        'schwimmend':1.0,
        'fliegend': 1.0,
        'schwebend': 1.0,
        'hüpfend': 1.0,
        'rutschend': 1.0
    }
    movement_metric = 0
    if movement is None:
        return 0
    for move in movement:
        move_type, move_distance = move.split()
        move_distance = int(move_distance)/5
        if move_type not in movement_multipliers:
            print(f"movement type {move_type} is currently not supported and set to 1")
            multiplier = 1.0
        else:
            multiplier = movement_multipliers.get(move_type.lower())
        movement_metric += move_distance * multiplier
    
    return movement_metric

def calculate_weaknesses_metric(weaknesses):
    weaknesses_metric = len(weaknesses)
    return weaknesses_metric

def calculate_resistances_metric(resistances):
    resistances_metric = len(resistances)
    return resistances_metric

def calculate_immunities_metric(immunities):
    immunities_metric = len(immunities)
    return immunities_metric

def calculate_ability_scores_metric(ability_scores):
    ability_scores_metric = 0
    ability_scores_metric = sum(ability_scores)
    return ability_scores_metric

def calculate_tank_metric(hp, ac):
    tank_metric = hp * ac / 100
    return tank_metric

def calculate_enemy_no_action_metric(row):
    tank_metric = calculate_tank_metric(row['hp'], row['ac'])
    movement_metric = calculate_movement_metric(row['movement'])
    weaknesses_metric = calculate_weaknesses_metric(row['weaknesses'])
    resistances_metric = calculate_resistances_metric(row['resistances'])
    immunities_metric = calculate_immunities_metric(row['immunities'])
    ability_scores = row[['str', 'dex', 'con', 'int', 'wis', 'cha']].values.tolist()
    ability_scores_metric = calculate_ability_scores_metric(ability_scores)

    no_action_metric = tank_metric + movement_metric + weaknesses_metric + resistances_metric + immunities_metric + ability_scores_metric
    return no_action_metric


enemies_df['no_action_metric'] = enemies_df.apply(calculate_enemy_no_action_metric, axis=1)
enemies_df


movement type krabbelnd is currently not supported and set to 1


,name,boss,bosstitle,type,hp,ac,movement,str,dex,con,int,wis,cha,weaknesses,resistances,immunities,actions,no_action_metric
0,Abwassernager,False,,Ungeziefer,30,15,"[laufend 40, schwimmend 30]",0,3,2,-3,-2,-1,[Feuer],[Säure],"[Gift, Krankheit]","[Rattenbiss, Pestwolke]",21.50
1,Abyssor,True,Peiniger des Meerestempels,Monstrosität,400,20,[schwimmend 100],7,2,6,2,3,4,[Blitz],"[Nicht-magischer Schaden, Kälte]","[Gift, Betäubt, Erschöpft, Bezaubert, Gepackt,...","[Reißzahnbiss, Schädelbrumme, Abyssaler Nebelh...",133.00
2,Aerocacti,False,,Pflanze,40,16,[fliegend 25],1,2,1,2,2,5,"[Feuer, Schnitt, Necrotisch]",[Stich],"[Gift, Acid]","[Stachelschuss, Stachelwalze]",30.40
3,Arachnia,False,,Bestie,40,18,[laufend 50],3,6,4,1,2,-4,"[Feuer, Blitz]",[Stich],"[Gift, Acid]","[Würgegriff, Netzschuss, Säurespeier]",34.20
4,Babasu,False,,Abscheulichkeit,55,16,[schwebend 30],1,3,2,5,4,6,[Heilig],[],[Nicht-magischer Schaden],"[Spektraler Biss, Fluch]",37.80
5,Bandagenwandler,False,,Untot,70,18,[laufend 10],6,-3,6,-1,2,-3,"[Heilig, Feuer]",[Schnitt],"[Gift, Acid]","[Würgegriff, Beilhieb]",26.60
6,Bandit,False,,Humanoid,80,17,[laufend 40],2,3,1,0,1,0,[],[],[],"[Schurkischer Angriff, Schneller Hieb, Blendgr...",28.60
7,Bestienkrieger,False,,Monstrosität,100,18,[laufend 30],6,1,5,-2,2,-1,[Feuer],[Nicht-magischer Schaden],[Verängstigt],"[Beilhieb, Brutale Hiebe, Zerquetschen, Berser...",38.00
8,Blutstahlritter,False,,Untot,200,20,[laufend 20],6,1,5,1,2,3,[Heilig],[Nicht-magischer Schaden],"[Gift, Nekrotisch, Verängstigt]","[Blutiger Schlag, Blutige Rache, Blutwoge]",67.00
9,Bohrschnabel,False,,Bestie,30,13,"[fliegend 50, laufend 10]",1,4,1,2,3,0,[Blitz],[],[],"[Bohrschnabel, Kreischen, Sturzflug]",27.90


In [6]:
from typing import Tuple, List


# Step 4: combine action-metric and enemy-metric to create danger level of an enemy
def calculate_max_action_potential(actions):
    # Get details for actions per enemy
    detailed_actions: List[Tuple[int, int, int, int]] = [] # contains list of tuple like this: [strength_metric,cooldown,max_dmg,avg_dmg]
    for action in actions:
        if action not in action_df["name"].values:
            print(f"{action} is not in action df")
            continue
        detailed_action_row = action_df[action_df["name"] == action]
        strength_metric = detailed_action_row["strength_metric"].iloc[0]
        cooldown = detailed_action_row["cooldown"].iloc[0]
        # if the cooldown duration type is not rounds (for example days) don't use the action for max action strength calculation
        if len(cooldown) == 2 and cooldown[1] == 'd':
            cooldown_round_duration = 10
        else:
            cooldown_round_duration = int(cooldown[0]) if len(cooldown) == 2 else 0    
        max_dmg = detailed_action_row["max_dmg"].iloc[0]
        avg_dmg = detailed_action_row["avg_dmg"].iloc[0]
        detailed_actions.append((strength_metric, cooldown_round_duration, max_dmg, avg_dmg))

    # Calculate max action potential by using averages for enemies with actions that have cooldown and fill the cooldown time with the second best option
    # Sort actions by strength_metric in descending order
    # detailed_actions = [(500,5,1,1),(400,2,1,1),(300,4,1,1),(100,0,1,1)]
    detailed_actions.sort(key=lambda x: x[0], reverse=True)
    strongest_action_cooldown_end = 0
    action_strength_sum = 0
    rounds = 0
    usable_actions = detailed_actions.copy()
    cooldown_list = [] # these are the actions that are on cooldown
    # tuple in usable_actions and cooldown_list list is build like this: [strength_metric,cooldown,max_dmg,avg_dmg]
    while True:
        # if the strongest action has no cooldown just stop after adding its strength else stop after the cooldown of the strongest action is up
        if strongest_action_cooldown_end < rounds:
            max_action_potential = action_strength_sum / rounds
            break
        
        # insert the weakest of the cooldown actions first at the beginning of the usable_list and after that the strongest, so the strongest is always the first element of the list
        cooldown_list.sort(key=lambda x: x[0], reverse=False)
        loop_postion = 0
        for strength_metric,cooldown,max_dmg,avg_dmg, on_cooldown_until in cooldown_list:
            if on_cooldown_until < rounds:
                usable_actions.insert(0,(strength_metric,cooldown,max_dmg,avg_dmg))
                del cooldown_list[loop_postion]
            else:   
                loop_postion =+ 1
        # if there are no actions left without a cooldown, then do nothing as an action if the rest is on cooldown        
        if len(usable_actions) == 0:
            usable_actions.insert(0,0,0,0)
        current_action_strength_metric = usable_actions[0][0]        
        current_action_cooldown = usable_actions[0][1]
        current_action_max_dmg = usable_actions[0][2]
        current_action_avg_dmg = usable_actions[0][3]
        on_cooldown_until = current_action_cooldown + rounds

        action_strength_sum += current_action_strength_metric
        # if the current action has cooldown add it to the cooldown_list and delete it from the usable_action list
        if current_action_cooldown != 0:
            if strongest_action_cooldown_end == 0:
                strongest_action_cooldown_end = on_cooldown_until
            cooldown_list.append((current_action_strength_metric, current_action_cooldown,current_action_max_dmg, current_action_avg_dmg, on_cooldown_until ))
            usable_actions = usable_actions[1:]
        rounds += 1

    # 0 + 3 = 3 (1,2,2,2,3) <- so after round 3 action number 2 is usable again (we start with round 0)
    return max_action_potential

def calculate_enemy_combined_metric(row):
    max_action_potential = calculate_max_action_potential(row["actions"])
    difficulty_rating = row["no_action_metric"] + max_action_potential
    return difficulty_rating

enemies_df['difficulty_rating'] = enemies_df.apply(calculate_enemy_combined_metric, axis=1)
enemies_df
# action potential = stärkste Action, wenn Action cooldown hat Action hinzufügen, dann auffüllen der danach stärksten Action bis cooldown erfüllt ist, dann average berechnen. Nuke(metric= 300, cooldown = 3r), Hieb(metric=100, cooldown=0r) -> action_potential = (300 + 100 + 100 + 100) / 4

,name,boss,bosstitle,type,hp,ac,movement,str,dex,con,int,wis,cha,weaknesses,resistances,immunities,actions,no_action_metric,difficulty_rating
0,Abwassernager,False,,Ungeziefer,30,15,"[laufend 40, schwimmend 30]",0,3,2,-3,-2,-1,[Feuer],[Säure],"[Gift, Krankheit]","[Rattenbiss, Pestwolke]",21.50,198.500000
1,Abyssor,True,Peiniger des Meerestempels,Monstrosität,400,20,[schwimmend 100],7,2,6,2,3,4,[Blitz],"[Nicht-magischer Schaden, Kälte]","[Gift, Betäubt, Erschöpft, Bezaubert, Gepackt,...","[Reißzahnbiss, Schädelbrumme, Abyssaler Nebelh...",133.00,660.500000
2,Aerocacti,False,,Pflanze,40,16,[fliegend 25],1,2,1,2,2,5,"[Feuer, Schnitt, Necrotisch]",[Stich],"[Gift, Acid]","[Stachelschuss, Stachelwalze]",30.40,50.400000
3,Arachnia,False,,Bestie,40,18,[laufend 50],3,6,4,1,2,-4,"[Feuer, Blitz]",[Stich],"[Gift, Acid]","[Würgegriff, Netzschuss, Säurespeier]",34.20,140.200000
4,Babasu,False,,Abscheulichkeit,55,16,[schwebend 30],1,3,2,5,4,6,[Heilig],[],[Nicht-magischer Schaden],"[Spektraler Biss, Fluch]",37.80,333.800000
5,Bandagenwandler,False,,Untot,70,18,[laufend 10],6,-3,6,-1,2,-3,"[Heilig, Feuer]",[Schnitt],"[Gift, Acid]","[Würgegriff, Beilhieb]",26.60,41.600000
6,Bandit,False,,Humanoid,80,17,[laufend 40],2,3,1,0,1,0,[],[],[],"[Schurkischer Angriff, Schneller Hieb, Blendgr...",28.60,46.963636
7,Bestienkrieger,False,,Monstrosität,100,18,[laufend 30],6,1,5,-2,2,-1,[Feuer],[Nicht-magischer Schaden],[Verängstigt],"[Beilhieb, Brutale Hiebe, Zerquetschen, Berser...",38.00,87.454545
8,Blutstahlritter,False,,Untot,200,20,[laufend 20],6,1,5,1,2,3,[Heilig],[Nicht-magischer Schaden],"[Gift, Nekrotisch, Verängstigt]","[Blutiger Schlag, Blutige Rache, Blutwoge]",67.00,124.750000
9,Bohrschnabel,False,,Bestie,30,13,"[fliegend 50, laufend 10]",1,4,1,2,3,0,[Blitz],[],[],"[Bohrschnabel, Kreischen, Sturzflug]",27.90,82.900000
